In [2]:
import time

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


device = torch.device("cuda")
print(device)

train_data = torchvision.datasets.CIFAR10(root="../dataset", train=True, transform=torchvision.transforms.ToTensor(),
                                          download=True)
test_data = torchvision.datasets.CIFAR10(root="../dataset", train=False, transform=torchvision.transforms.ToTensor(),
                                         download=True)


train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class MyModule(nn.Module):
    def __init__(self):
        super(MyModule, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, 2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64 * 4 * 4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.model(x)
        return x


mymodel = MyModule()
mymodel.to(device)

# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn.to(device)

learning_rate = 1e-2
optimizer = torch.optim.SGD(mymodel.parameters(), lr=learning_rate)

total_train_step = 0

total_test_step = 0

epoch = 10

# 添加tensorboard
writer = SummaryWriter("../logs_train")

start_time = time.time()
for i in range(epoch):
    print("-------第 {} 轮训练开始-------".format(i + 1))

    # 训练步骤开始
    mymodel.train()
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        outputs = mymodel(imgs)
        loss = loss_fn(outputs, targets)

        # 优化器优化模型
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step = total_train_step + 1
        if total_train_step % 100 == 0:
            end_time = time.time()
            print(end_time - start_time)
            print("训练次数：{}, Loss: {}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)

    # 测试步骤开始
    mymodel.eval()
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device)
            outputs = mymodel(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss = total_test_loss + loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            total_accuracy = total_accuracy + accuracy

    print("整体测试集上的Loss: {}".format(total_test_loss))
    print("整体测试集上的正确率: {}".format(total_accuracy / test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_accuracy / test_data_size, total_test_step)
    total_test_step = total_test_step + 1

    torch.save(mymodel, "mymodel_{}.pth".format(i))
    print("模型已保存")

writer.close()



cuda
Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000
-------第 1 轮训练开始-------
2.714474678039551
训练次数：100, Loss: 2.288367986679077
4.1766345500946045
训练次数：200, Loss: 2.2803854942321777
5.1956000328063965
训练次数：300, Loss: 2.2535040378570557
6.26540732383728
训练次数：400, Loss: 2.19071888923645
7.279852390289307
训练次数：500, Loss: 2.0880324840545654
8.30460786819458
训练次数：600, Loss: 2.033418655395508
9.600167512893677
训练次数：700, Loss: 1.9868113994598389
整体测试集上的Loss: 310.50523853302
整体测试集上的正确率: 0.29739999771118164
模型已保存
-------第 2 轮训练开始-------
12.34154725074768
训练次数：800, Loss: 1.87624192237854
13.366432428359985
训练次数：900, Loss: 1.82535982131958
14.620078086853027
训练次数：1000, Loss: 1.9706889390945435
16.20315384864807
训练次数：1100, Loss: 2.0024752616882324
17.305925130844116
训练次数：1200, Loss: 1.7099025249481201
18.366520166397095
训练次数：1300, Loss: 1.6563485860824585
19.40901255607605
训练次数：1400, Loss: 1.7519627809524536
20.470015287399292
训练次数：1500,